In [1]:
!pip install requests

In [2]:
# Import Necessary Modules
import requests
import json as jn

### API certification

In [3]:
# Function to add certification (MPAA Rating) to movie information
def add_certification(movie_info):
    movie_id = movie_info['id']
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=e6439afc327b0d52d1d2b3517a3493f8"
    response = requests.get(url)
    data = jn.loads(response.content.decode('utf-8'))
    certification = data['release_dates']['results'][0]['certification']
    movie_info['certification'] = certification
    return movie_info

print("Certification Completed")

Certification Completed


### Function to Append/Extend

In [4]:
import json as jn

# Function to append data to a JSON file
def append_json(file_path, data):
    with open(file_path, 'r+') as file:
        file_data = jn.load(file)
        file_data.extend(data)
        file.seek(0)
        jn.dump(file_data, file, indent=4)

print("Append done")

Append done


- **Example**

In [5]:
# Example usage
movie_information = {
    'id': 'tt0848228',  # Example movie ID
    'title': 'The Avengers'  # Example movie title
}
print("Started")
updated_movie_info = add_certification(movie_information)
print(">....")
append_json('movies.json', [updated_movie_info])
print(">..")
print("Done")

Started


KeyError: 'release_dates'

In [ ]:
import pandas as pd
import json
import requests

# Function to add certification (MPAA Rating) to movie information
def add_certification(movie_info):
    movie_id = movie_info['id']
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=e6439afc327b0d52d1d2b3517a3493f8"
    response = requests.get(url)
    data = json.loads(response.content.decode('utf-8'))
    certification = data['release_dates']['results'][0]['certification']
    movie_info['certification'] = certification
    return movie_info

# Function to append data to a JSON file
def append_json(file_path, data):
    with open(file_path, 'a') as file:
        json.dump(data, file)
        file.write('\n')

# Load the filtered movie data from Part 1
filtered_movies = pd.read_csv('filtered_movies.csv.gz')

# Iterate over each movie in the filtered dataset
for _, movie in filtered_movies.iterrows():
    movie_info = {
        'id': movie['imdb_id'],
        'title': movie['title']
    }
    updated_movie_info = add_certification(movie_info)
    append_json('movies.json', updated_movie_info)

print("Extraction and appending completed.")